# ⚡ RAG Embedding + FAISS Index (Colab GPU)
This notebook loads customer support data, encodes queries using `sentence-transformers`, caches embeddings, builds a FAISS index, and supports basic GPT-2-based RAG querying.

In [ ]:
# 📦 Install dependencies
!pip install -q faiss-cpu sentence-transformers datasets transformers

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

In [ ]:
import numpy as np
import hashlib
from tqdm import tqdm
import os
import faiss
import pandas as pd
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import torch

# Cache paths
CACHE_DIR = "cache"
DATA_CACHE_FILE = os.path.join(CACHE_DIR, "customer_support_full.csv")
os.makedirs(CACHE_DIR, exist_ok=True)

# Load or cache dataset
if os.path.exists(DATA_CACHE_FILE):
    print(f"🔁 Loading dataset from cache: {DATA_CACHE_FILE}")
    df = pd.read_csv(DATA_CACHE_FILE)
else:
    print("🌐 Downloading dataset...")
    dataset = load_dataset("MohammadOthman/mo-customer-support-tweets-945k")
    df = pd.DataFrame({
        "customer_query": dataset["train"]["input"],
        "support_reply": dataset["train"]["output"]
    })
    #df = df.head(10000)  # Use subset
    df.to_csv(DATA_CACHE_FILE, index=False)
    print(f"✅ Cached dataset to {DATA_CACHE_FILE}")

# Embedding and FAISS setup
class VectorStore:
    def __init__(self, data, batch_size=128, cache_dir="cache/"):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)

        self.cache_file = os.path.join(cache_dir, "embeddings_full.npy")

        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🚀 Using device: {self.device}")
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device=self.device)

        if os.path.exists(self.cache_file):
            print(f"🔁 Loading cached embeddings from {self.cache_file}...")
            self.embeddings = np.load(self.cache_file)
            print(f"✅ Loaded embeddings from cache.")
        else:
            print("⚙️ Generating embeddings...")
            self.data = data
            self.index = faiss.IndexFlatL2(384)

            self.embeddings = []
            for i in tqdm(range(0, len(data), batch_size)):
                batch = data['customer_query'].iloc[i:i+batch_size].tolist()
                encoded = self.model.encode(batch, show_progress_bar=False)
                self.embeddings.extend(encoded)

            self.embeddings = np.array(self.embeddings)
            np.save(self.cache_file, self.embeddings)
            print(f"✅ Saved embeddings to {self.cache_file}")

        self.data = data
        self.index = faiss.IndexFlatL2(384)
        self.index.add(self.embeddings)

    def search(self, query, top_k=3):
        query_vec = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_vec), top_k)
        valid_indices = [i for i in indices[0] if 0 <= i < len(self.data)]
        return [self.data.iloc[i]['support_reply'] for i in valid_indices]

# 🧪 Instantiate VectorStore and test
vector_store = VectorStore(df)
print(vector_store.search("I ordered a laptop, but it arrived with a broken screen. What should I do?"))



In [ ]:
!huggingface-cli login

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import time
import pandas as pd
import os
from datetime import datetime

# Use LLaMA 2 7B Chat model (make sure you have access)
model_id = "meta-llama/Llama-2-7b-chat-hf"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

print(model.hf_device_map)

# Pipeline setup
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# CSV log setup
LOG_PATH = "chat_log.csv"
if not os.path.exists(LOG_PATH):
    pd.DataFrame(columns=["timestamp", "query", "context", "response"]).to_csv(LOG_PATH, index=False)

def log_interaction(query, context, response):
    timestamp = datetime.now().isoformat()
    new_entry = {
        "timestamp": timestamp,
        "query": query,
        "context": context,
        "response": response
    }
    log_df = pd.read_csv(LOG_PATH)
    log_df = pd.concat([log_df, pd.DataFrame([new_entry])], ignore_index=True)
    log_df.to_csv(LOG_PATH, index=False)
    print(f"📝 Interaction logged at {timestamp}")

def generate_response(query):
    context = vector_store.search(query)
    context_str = "\n".join(context)

    prompt = f"""<s>[INST] <<SYS>>You are a helpful support assistant.<</SYS>>

    Past replies:
    {context_str}

    New question: {query}
    Answer: [/INST]"""

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    start = time.time()
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )
    end = time.time()

    print(f"⏱️ Time taken: {end - start:.2f} seconds")
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("Answer:")[-1].strip()
    print("💬 Response:", answer)

    # Log the interaction
    log_interaction(query, context_str, answer)

    return answer


In [ ]:
# 🚀 Try it out!
generate_response("I ordered a laptop, but it arrived with a broken screen. What should I do?")

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn
from threading import Thread

# Allow nested async loops (for running Uvicorn in a Jupyter notebook)
nest_asyncio.apply()

# Define FastAPI app
app = FastAPI()

# Define request schema for the incoming query
class QueryRequest(BaseModel):
    query: str

# Function to generate a response based on your RAG model
#def generate_response(query: str):
    # Replace this with your actual RAG model logic or vector store search
    # This is where you would query your vector store and use the LLM to generate a response
    #return f"You can reset your password by following the instructions provided in the email sent to you."

# Define FastAPI route to handle the query
@app.post("/rag-query")
async def rag_query(request: QueryRequest):
    query = request.query
    # Use the generate_response function to get a response based on the query
    response = generate_response(query)
    return {"results": [response]}

# Start the server in a thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start the FastAPI server in a separate thread to avoid blocking
Thread(target=run).start()


In [ ]:
#Please copy and paste your ngrok Authtoken in place of the text
os.environ["NGROK_AUTHTOKEN"] = "Put your token here"

In [ ]:
import os

# Get authtoken from environment variable
authtoken = os.getenv("NGROK_AUTHTOKEN")

if authtoken:
    conf.get_default().auth_token = authtoken
else:
    raise ValueError("🚫 NGROK_AUTHTOKEN environment variable is not set!")

In [ ]:
conf.get_default().auth_token = authtoken

# Start ngrok tunnel and print the public URL
public_url = ngrok.connect(8000).public_url
print(f"🌐 Public URL: {public_url}")

In [ ]:
import requests

# Using ngrok public url to process rag query
url = f"{public_url}/rag-query"

# Payload with the query you want to send
payload = {"query": "I ordered a laptop, but it arrived with a broken screen. What should I do?"}

# Sending the POST request to the FastAPI endpoint
response = requests.post(url, json=payload)

# Print the response from the FastAPI server (the result of the query)
print(response.json())

In [ ]:

# Define your queries (you can modify these or dynamically pass them based on user input)
queries = [
    "I need help resetting my password.",
    "I didn’t receive the reset link."
]

# Function to handle multi-turn queries
def get_rag_response(query_list):
    response = None
    for query in query_list:
        payload = {"query": query}
        response = requests.post(url, json=payload)
        if response.status_code == 200:
            print(f"Response to '{query}': {response.json()}")
        else:
            print(f"Error in response: {response.status_code}")
    return response

# Call the function to process the queries
get_rag_response(queries)

In [ ]:
# Payload with the query you want to send
payload = {"query": "My cat chewed my phone charger. Is this covered under warranty?"}

# Sending the POST request to the FastAPI endpoint
response = requests.post(url, json=payload)

# Print the response from the FastAPI server (the result of the query)
print(response.json())

In [ ]:
# Payload with the query you want to send
payload = {"query": "Why did you suggest contacting support?"}

# Sending the POST request to the FastAPI endpoint
response = requests.post(url, json=payload)

# Print the response from the FastAPI server (the result of the query)
print(response.json())